## 1. Data

#### 1.1. Import libraries and data

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as  pd, numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [3]:
import funciones as fun
import variables_nombres as vn

In [4]:
path = r'..\..\output\data_preprocess\dfs_0_i_pca.csv'
data = pd.read_csv( path )

In [5]:
# Borrar otras variables dependientes. Solo nos quedamos con 'monto_corrup2'
data = data.drop( [ 'monto_examinado', 'monto_objeto_servicio', 'corrup_intensa', 
                    'corrup_amplia', 'per_corrup1', 'monto_corrup2' ], 
                  axis = 1 )

In [6]:
data = data.dropna( axis = 1 )

#### 1.2. Split data into training and test set

In [7]:
np.random.seed( 1234 )
training = np.random.choice( data.index, size=int( len( data )*( 3/4 )), replace = False )

data_train = data.loc[ training, : ]
data_test = data.drop( training, axis = 0 )

## 2. Definir el modelo

In [8]:
import statsmodels.formula.api as smf
import patsy

In [9]:
#varnames =  data.columns.values.tolist()
#varnames = pd.DataFrame( varnames )
#varnames.to_excel('varnames_pca.xlsx')

#### 2.1. Construir modelos

In [10]:
%%time

formula_basic = "per_corrup2 ~ tejgfun_ct05pgercon + devppimfun_ct05pgercon + devppimfun_ct05opseg + devppimfun_ct05trab + piagfun_ct05come + devppimfun_ct05turi + tejgfun_ct05agro + tdvgfun_ct05agro + devppimfun_ct05agro + piagfun_ct05energia + tejgfun_ct05energia + pimgfun_ct05ind + pimgfun_ct05trans + tejgfun_ct05trans + devppimfun_ct05trans + devppimfun_ct05amb + tejgfun_ct05san + devppimfun_ct05san + devppimfun_ct05viv + tejgfun_ct05protsoc + devppimfun_ct05protsoc + tejgfun_ct05prevsoc + devppimfun_ct05dpub + tejgfun_ct06pgercon + devppimfun_ct06pgercon + piagfun_ct06opseg + pimgfun_ct06opseg + tejgfun_ct06opseg + devppimfun_ct06opseg + devppimfun_ct06trab + tejgfun_ct06come + piagfun_ct06turi + pimgfun_ct06turi + tdvgfun_ct06turi + devppimfun_ct06turi + pimgfun_ct06ind + tejgfun_ct06ind + devppimfun_ct06trans + pimgfun_ct06amb + piagfun_ct06san + tdvgfun_ct06san + devppimfun_ct06san + tejgfun_ct06viv + devppimfun_ct06viv + devppimfun_ct06salud + devppimfun_ct06edu + tejgfun_ct06protsoc + devppimfun_ct06protsoc + devppimft_rdet + piagrb_reod + tejgrb_reod + devppimrb_reod + piagrb_dotr + tejgrb_dotr + tejgrb_fonc + devppimrb_fonc + devppimrb_canr + devppimct_r00gstcr + tejgct_r13gstcr + tdvgct_r13gstcr + devppimct_r07srdeu + pimgge_r00ct05pobso + devppimge_r00ct05pobso + devppimge_r00ct05popso + tejgge_r00ct05biser + devppimge_r00ct05biser + pimgge_r00ct05otgst + devppimgge_r00ct05otgst + devppimgge_r00ct06dotra + devppimgge_r00ct06otgst + tdvgge_r00ct06acanf + devppimgge_r09ct05pobso + tejgge_r09ct05popso + devppimgge_r09ct05popso + devppimgge_r09ct05biser + pimgge_r09ct05dotra + tejgge_r09ct05dotra + devppimgge_r09ct05dotra + piagge_r09ct05otgst + tdvgge_r09ct05otgst + devppimgge_r09ct05otgst + piagge_r09ct06dotra + pimgge_r09ct06dotra + devppimgge_r09ct06dotra + piagge_r09ct06otgst + tejgge_r09ct06otgst + devppimgge_r09ct06otgst + tejgge_r09ct06acanf + devppimgge_r09ct06acanf + dfgpimpiagge_r09ct07sedpu + devppimgge_r19ct05biser + dfgpimpiagge_r19ct06dotra + dfgdevpiagge_r19ct06dotra + devppimgge_r19ct06dotra + dfgpimpiagge_r13ct05pobso + tejgge_r13ct05popso + dfgpimpiagge_r13ct05popso + devppimgge_r13ct05popso + devppimgge_r13ct05biser + piagge_r13ct05dotra + tejgge_r13ct05dotra + devppimgge_r13ct05dotra + pimgge_r13ct05otgst + devppimgge_r13ct05otgst + dfgpimpiagge_r13ct06dotra + devppimgge_r13ct06dotra + pimgge_r07ct05pobso + tejgge_r07ct05pobso + devppimgge_r07ct05pobso + piagge_r07ct05popso + devppimgge_r07ct05popso + tdvgge_r07ct05biser + devppimgge_r07ct05biser + pimgge_r07ct05dotra + tejgge_r07ct05dotra + devppimgge_r07ct05dotra + pimgge_r07ct05otgst + tejgge_r07ct05otgst + devppimgge_r07ct05otgst + devppimge_r08ct05pobso + piagge_r08ct05popso + devppimge_r08ct05popso + piagge_r08ct05dotra + tejgge_r08ct05dotra + tdvgge_r08ct05dotra + devppimge_r08ct05dotra + devppimgge_r08ct05otgst + devppimgge_r08ct06acanf + piagge_r08ct07sedpu + pimgge_r08ct07sedpu + devppimgge_r08ct07sedpu + pimgge_r18ct05pobso + devppimge_r18ct05pobso + pimgge_r18ct05popso + devppimge_r18ct05popso + devppimge_r18ct05biser + tdvgge_r18ct05dotra + devppimge_r18ct05dotra + piagge_r18ct05otgst + tejgge_r18ct05otgst + devppimgge_r18ct05otgst + dfgpimpiagge_r18ct06acacf + piagge_r18ct07sedpu + piagkft_reod + dfgpimpiagkft_reod + devppimgkft_reod + tdvgkft_redr + dfgpimpiagkft_rooc + devppimgkft_rooc + devppimgkft_dotr + devppimgkft_rdet + piagkftr07_rdet + devppimgkftr07_rdet + tejgfun_f1ct05agro + tejgfun_f1ct05amb + tejgfun_f1ct05come + tejgfun_f1ct05cydep + tdvgfun_f1ct05cydep + devppimfun_f1ct05cydep + devppimfun_f1ct05edu + tejgfun_f1ct05opseg + dfgpimpiafun_f1ct05opseg + devppimfun_f1ct05opseg + devppimfun_f1ct05pgercon + tejgfun_f1ct05salud + dfgpimpiafun_f1ct05salud + tejgfun_f1ct05san + dfgdevpiagfun_f1ct05san + devppimfun_f1ct05san + piagfun_f1ct05trans + tejgfun_f1ct05trans + dfgpimpiafun_f1ct05trans + dfgdevpiagfun_f1ct05trans + devppimfun_f1ct05trans + tejgfun_f1ct05viv + dfgpimpiafun_f1ct05viv + devppimfun_f1ct05viv + devppimfun_f1ct06agro + devppimfun_f1ct06amb + dfgdevpiagfun_f1ct06come + tdvgfun_f1ct06cydep + devppimfun_f1ct06cydep + piagfun_f1ct06edu + tejgfun_f1ct06opseg + dfgdevpiagfun_f1ct06opseg + devppimfun_f1ct06opseg + dfgpimpiafun_f1ct06pgercon + devppimfun_f1ct06pgercon + tejgfun_f1ct06protsoc + dfgpimpiafun_f1ct06protsoc + devppimfun_f1ct06protsoc + dfgpimpiafun_f1ct06salud + devppimfun_f1ct06salud + tejgfun_f1ct06trans + tejgfun_f1ct06viv + devppimfun_f1ct06viv + piagfun_f2ct05agro + tdvgfun_f2ct05agro + devppimfun_f2ct05agro + devppimfun_f2ct05amb + tejgfun_f2ct05come + devppimfun_f2ct05cydep + piagfun_f2ct05edu + pimgfun_f2ct05edu + devppimfun_f2ct05edu + tejgfun_f2ct05energia + tdvgfun_f2ct05energia + devppimfun_f2ct05energia + piagfun_f2ct05ind + devppimfun_f2ct05opseg + piagfun_f2ct05protsoc + devppimfun_f2ct05protsoc + piagfun_f2ct05san + pimgfun_f2ct05san + tejgfun_f2ct05san + devppimfun_f2ct05san + piagfun_f2ct05trans + devppimfun_f2ct05trans + tejgfun_f2ct05viv + piagfun_f2ct06agro + devppimfun_f2ct06agro + piagfun_f2ct06amb + tejgfun_f2ct06amb + devppimfun_f2ct06amb + piagfun_f2ct06come + pimgfun_f2ct06come + tdvgfun_f2ct06come + devppimfun_f2ct06come + tejgfun_f2ct06comunica + dfgpimpiafun_f2ct06comunica + pimgfun_f2ct06cydep + tejgfun_f2ct06cydep + devppimfun_f2ct06cydep + devppimfun_f2ct06edu + piagfun_f2ct06energia + tejgfun_f2ct06ind + devppimfun_f2ct06ind + pimgfun_f2ct06opseg + tejgfun_f2ct06opseg + devppimfun_f2ct06opseg + piagfun_f2ct06pgercon + pimgfun_f2ct06pgercon + tejgfun_f2ct06pgercon + devppimfun_f2ct06pgercon + piagfun_f2ct06protsoc + pimgfun_f2ct06protsoc + tejgfun_f2ct06protsoc + devppimfun_f2ct06protsoc + piagfun_f2ct06salud + tejgfun_f2ct06salud + devppimfun_f2ct06salud + piagfun_f2ct06san + tejgfun_f2ct06san + devppimfun_f2ct06san + piagfun_f2ct06trans + pimgfun_f2ct06trans + devppimfun_f2ct06trans + piagfun_f2ct06turi + pimgfun_f2ct06turi + tejgfun_f2ct06turi + tdvgfun_f2ct06turi + devppimfun_f2ct06turi + pimgfun_f2ct06viv + tejgfun_f2ct06viv + devppimfun_f2ct06viv + devppimfun_f3ct05opseg + tejgfun_f3ct05pgercon + tdvgfun_f3ct05pgercon + tdvgfun_f3ct06opseg + dfgpimpiafun_f3ct06opseg + tejgfun_f3ct06pgercon + devppimfun_f3ct06pgercon + dfgpimpiafun_f4ct05agro + devppimfun_f4ct05agro + piagfun_f4ct05amb + devppimfun_f4ct05amb + dfgpimpiafun_f4ct05come + devppimfun_f4ct05come + tdvgfun_f4ct05cydep + devppimfun_f4ct05cydep + dfgpimpiafun_f4ct05edu + dfgpimpiafun_f4ct05opseg + devppimfun_f4ct05opseg + pimgfun_f4ct05pgercon + tdvgfun_f4ct05pgercon + tejgfun_f4ct05protsoc + tejgfun_f4ct05salud + dfgpimpiafun_f4ct05salud + devppimfun_f4ct05salud + dfgpimpiafun_f4ct05san + tejgfun_f4ct05trans + devppimfun_f4ct05trans + tejgfun_f4ct05turi + dfgpimpiafun_f4ct05turi + devppimfun_f4ct05turi + tejgfun_f4ct05viv + dfgpimpiafun_f4ct05viv + devppimfun_f4ct05viv + tejgfun_f4ct06agro + tdvgfun_f4ct06agro + devppimfun_f4ct06amb + devppimfun_f4ct06come + devppimfun_f4ct06cydep + tejgfun_f4ct06edu + dfgpimpiafun_f4ct06edu + devppimfun_f4ct06edu + piagfun_f4ct06pgercon + dfgpimpiafun_f4ct06pgercon + devppimfun_f4ct06pgercon + dfgpimpiafun_f4ct06protsoc + devppimfun_f4ct06protsoc + dfgpimpiafun_f4ct06salud + devppimfun_f4ct06salud + dfgpimpiafun_f4ct06san + devppimfun_f4ct06trab + devppimfun_f4ct06turi + dfgpimpiafun_f4ct06viv + piagfun_f5ct05agro + tejgfun_f5ct05agro + devppimfun_f5ct05come + devppimfun_f5ct05comunica + devppimfun_f5ct05cydep + dfgdevpiagfun_f5ct05dpub + tejgfun_f5ct05edu + tdvgfun_f5ct05edu + devppimfun_f5ct05edu + tejgfun_f5ct05energia + tdvgfun_f5ct05energia + devppimfun_f5ct05energia + pimgfun_f5ct05opseg + tejgfun_f5ct05opseg + piagfun_f5ct05pgercon + devppimfun_f5ct05pgercon + devppimfun_f5ct05protsoc + piagfun_f5ct05salud + tejgfun_f5ct05salud + devppimfun_f5ct05salud + piagfun_f5ct05san + devppimfun_f5ct05trab + piagfun_f5ct05turi + tejgfun_f5ct05viv + devppimfun_f5ct06amb + pimgfun_f5ct06come + devppimfun_f5ct06come + devppimfun_f5ct06comunica + devppimfun_f5ct06cydep + piagfun_f5ct06edu + pimgfun_f5ct06protsoc + pimgfun_f5ct06trab + tejgfun_f5ct06trab + devppimfun_f5ct06trans + pimgfun_f5energia + devppimfun_f5energia + piagfun_f5r07ct05agro + tejgfun_f5r07ct05agro + devppimfun_f5r07ct05agro + piagfun_f5r07ct05comunica + tejgfun_f5r07ct05cydep + tdvgfun_f5r07ct05cydep + devppimfun_f5r07ct05cydep + dfgdevpiagfun_f5r07ct05dpub + piagfun_f5r07ct05edu + tejgfun_f5r07ct05edu + devppimfun_f5r07ct05edu + tejgfun_f5r07ct05energia + tdvgfun_f5r07ct05energia + devppimfun_f5r07ct05energia + piagfun_f5r07ct05ind + devppimfun_f5r07ct05ind + devppimfun_f5r07ct05opseg + devppimfun_f5r07ct05pgercon + devppimfun_f5r07ct05protsoc + tdvgfun_f5r07ct05salud + piagfun_f5r07ct05san + tejgfun_f5r07ct05san + devppimfun_f5r07ct05san + piagfun_f5r07ct05trans + tejgfun_f5r07ct05trans + tdvgfun_f5r07ct05trans + devppimfun_f5r07ct05trans + piagfun_f5r07ct05turi + pimgfun_f5r07ct05turi + tejgfun_f5r07ct05turi + devppimfun_f5r07ct05turi + pimgfun_f5r07ct05viv + tejgfun_f5r07ct05viv + devppimfun_f5r07ct05viv + tejgfun_f5r07ct06agro + tdvgfun_f5r07ct06agro + devppimfun_f5r07ct06agro + piagfun_f5r07ct06amb + pimgfun_f5r07ct06amb + tejgfun_f5r07ct06amb + devppimfun_f5r07ct06amb + pimgfun_f5r07ct06come + tejgfun_f5r07ct06come + tdvgfun_f5r07ct06come + devppimfun_f5r07ct06come + devppimfun_f5r07ct06comunica + piagfun_f5r07ct06cydep + pimgfun_f5r07ct06cydep + tejgfun_f5r07ct06cydep + devppimfun_f5r07ct06cydep + tejgfun_f5r07ct06edu + devppimfun_f5r07ct06edu + piagfun_f5r07ct06energia + devppimfun_f5r07ct06energia + tdvgfun_f5r07ct06ind + piagfun_f5r07ct06opseg + tejgfun_f5r07ct06opseg + devppimfun_f5r07ct06opseg + pimgfun_f5r07ct06pgercon + tejgfun_f5r07ct06pgercon + devppimfun_f5r07ct06pgercon + devppimfun_f5r07ct06prevsoc + tejgfun_f5r07ct06protsoc + tdvgfun_f5r07ct06protsoc + devppimfun_f5r07ct06protsoc + piagfun_f5r07ct06salud + devppimfun_f5r07ct06salud + devppimfun_f5r07ct06trab + devppimfun_f5r07ct06trans + tejgfun_f5r07ct06turi + tdvgfun_f5r07ct06turi + devppimfun_f5r07ct06turi + devppimfun_f5r07ct06viv + tdvgfun_f5r08ct05agro + devppimfun_f5r08ct05agro + devppimfun_f5r08ct05amb + piagfun_f5r08ct05come + devppimfun_f5r08ct05comunica + devppimfun_f5r08ct05cydep + piagfun_f5r08ct05edu + tejgfun_f5r08ct05edu + tdvgfun_f5r08ct05edu + devppimfun_f5r08ct05edu + piagfun_f5r08ct05energia + tejgfun_f5r08ct05energia + devppimfun_f5r08ct05energia + pimgfun_f5r08ct05opseg + devppimfun_f5r08ct05opseg + devppimfun_f5r08ct05pgercon + devppimfun_f5r08ct05protsoc + piagfun_f5r08ct05salud + tejgfun_f5r08ct05salud + piagfun_f5r08ct05san + tejgfun_f5r08ct05san + tdvgfun_f5r08ct05san + devppimfun_f5r08ct05san + devppimfun_f5r08ct05trans + piagfun_f5r08ct06agro + devppimfun_f5r08ct06agro + piagfun_f5r08ct06amb + pimgfun_f5r08ct06amb + tejgfun_f5r08ct06amb + devppimfun_f5r08ct06amb + pimgfun_f5r08ct06come + tejgfun_f5r08ct06come + tdvgfun_f5r08ct06come + devppimfun_f5r08ct06come + piagfun_f5r08ct06comunica + tdvgfun_f5r08ct06comunica + devppimfun_f5r08ct06comunica + piagfun_f5r08ct06cydep + pimgfun_f5r08ct06cydep + tejgfun_f5r08ct06cydep + devppimfun_f5r08ct06cydep + piagfun_f5r08ct06edu + devppimfun_f5r08ct06edu + piagfun_f5r08ct06energia + devppimfun_f5r08ct06energia + pimgfun_f5r08ct06opseg + tejgfun_f5r08ct06opseg + devppimfun_f5r08ct06opseg + piagfun_f5r08ct06pgercon + tejgfun_f5r08ct06pgercon + devppimfun_f5r08ct06pgercon + piagfun_f5r08ct06protsoc + pimgfun_f5r08ct06protsoc + tejgfun_f5r08ct06protsoc + tdvgfun_f5r08ct06protsoc + devppimfun_f5r08ct06protsoc + tdvgfun_f5r08ct06salud + devppimfun_f5r08ct06salud + tejgfun_f5r08ct06san + tdvgfun_f5r08ct06san + devppimfun_f5r08ct06san + dfgpimpiafun_f5r08ct06trab + tejgfun_f5r08ct06trans + devppimfun_f5r08ct06trans + piagfun_f5r08ct06turi + tejgfun_f5r08ct06turi + devppimfun_f5r08ct06turi + piagfun_f5r08ct06viv + tejgfun_f5r08ct06viv + tdvgfun_f5r08ct06viv + devppimfun_f5r08ct06viv + tdvgtotfun_f5r18comunica + piagfun_f5r18ct05agro + pimgfun_f5r18ct05agro + devppimfun_f5r18ct05agro + piagfun_f5r18ct05amb + tejgfun_f5r18ct05amb + tdvgfun_f5r18ct05amb + devppimfun_f5r18ct05amb + piagfun_f5r18ct05come + pimgfun_f5r18ct05come + devppimfun_f5r18ct05come + tdvgfun_f5r18ct05cydep + devppimfun_f5r18ct05cydep + pimgfun_f5r18ct05edu + tejgfun_f5r18ct05edu + devppimfun_f5r18ct05edu + piagfun_f5r18ct05energia + devppimfun_f5r18ct05energia + tejgfun_f5r18ct05ind + devppimfun_f5r18ct05ind + piagfun_f5r18ct05opseg + tejgfun_f5r18ct05opseg + devppimfun_f5r18ct05opseg + dfgdevpiagfun_f5r18ct05pesca + devppimfun_f5r18ct05pgercon + tejgfun_f5r18ct05prevsoc + piagfun_f5r18ct05protsoc + pimgfun_f5r18ct05protsoc + tejgfun_f5r18ct05protsoc + devppimfun_f5r18ct05protsoc + devppimfun_f5r18ct05salud + piagfun_f5r18ct05san + devppimfun_f5r18ct05san + devppimfun_f5r18ct05trab + piagfun_f5r18ct05trans + tejgfun_f5r18ct05trans + devppimfun_f5r18ct05trans + piagfun_f5r18ct05turi + tejgfun_f5r18ct05turi + tdvgfun_f5r18ct05turi + devppimfun_f5r18ct05turi + piagfun_f5r18ct05viv + pimgfun_f5r18ct05viv + tejgfun_f5r18ct05viv + tdvgfun_f5r18ct05viv + devppimfun_f5r18ct05viv + devppimfun_f5r18ct06agro + tdvgfun_f5r18ct06amb + devppimfun_f5r18ct06amb + devppimfun_f5r18ct06come + devppimfun_f5r18ct06ind + devppimfun_f5r18ct06opseg + pimgfun_f5r18ct06pesca + devppimfun_f5r18ct06pesca + piagfun_f5r18ct06pgercon + devppimfun_f5r18ct06pgercon + piagfun_f5r18ct06prevsoc + devppimfun_f5r18ct06prevsoc + devppimfun_f5r18ct06protsoc + devppimfun_f5r18ct06salud + dfgpimpiafun_f5r18ct06trab + devppimfun_f5r18ct06trab + devppimfun_f5r18ct06viv + piagtotfun_f1agro + tejgtotfun_f1agro + tdvgtotfun_f1agro + devppimtotfun_f1agro + piagtotfun_f1amb + devppimtotfun_f1amb + tejgtotfun_f1come + dfgdevpiagtotfun_f1come + devppimtotfun_f1come + tejgtotfun_f1cydep + piagtotfun_f1edu + pimgtotfun_f1edu + tejgtotfun_f1edu + devppimtotfun_f1edu + dfgdevpiagtotfun_f1energia + devppimtotfun_f1energia + tejgtotfun_f1pgercon + piagtotfun_f1prevsoc + tdvgtotfun_f1prevsoc + devppimtotfun_f1prevsoc + piagtotfun_f1salud + tejgtotfun_f1salud + devppimtotfun_f1salud + piagtotfun_f1san + tdvgtotfun_f1san + devppimtotfun_f1san + dfgpimpiatotfun_f1trab + dfgpimpiatotfun_f1trans + devppimtotfun_f1trans + tdvgtotfun_f1turi + dfgpimpiatotfun_f1turi + devppimtotfun_f1turi + pimgtotfun_f1viv + piagtotfun_f2agro + tejgtotfun_f2agro + tdvgtotfun_f2agro + devppimtotfun_f2agro + piagtotfun_f2comunica + pimgtotfun_f2cydep + tejgtotfun_f2cydep + dfgpimpiatotfun_f2dpub + devppimtotfun_f2dpub + piagtotfun_f2edu + tejgtotfun_f2edu + devppimtotfun_f2edu + piagtotfun_f2energia + devppimtotfun_f2energia + devppimtotfun_f2ind + tejgtotfun_f2pgercon + devppimtotfun_f2pgercon + devppimtotfun_f2prevsoc + tejgtotfun_f2protsoc + pimgtotfun_f2salud + devppimtotfun_f2salud + piagtotfun_f2san + tejgtotfun_f2san + devppimtotfun_f2san + dfgpimpiatotfun_f2trab + tejgtotfun_f2trans + devppimtotfun_f2trans + piagtotfun_f2turi + pimgtotfun_f2turi + tdvgtotfun_f2turi + devppimtotfun_f2turi + tdvgtotfun_f2viv + devppimtotfun_f2viv + piagtotfun_f3agro + pimgtotfun_f3agro + tejgtotfun_f3agro + devppimtotfun_f3agro + dfgdevpiagtotfun_f3amb + devppimtotfun_f3amb + devppimtotfun_f3come + dfgpimpiatotfun_f3comunica + tejgtotfun_f3cydep + dfgpimpiatotfun_f3cydep + devppimtotfun_f3cydep + piagtotfun_f3edu + tejgtotfun_f3edu + devppimtotfun_f3edu + tejgtotfun_f3energia + dfgdevpiagtotfun_f3energia + devppimtotfun_f3energia + dfgpimpiatotfun_f3ind + tejgtotfun_f3opseg + tdvgtotfun_f3opseg + devppimtotfun_f3opseg + dfgpimpiatotfun_f3pesca + piagtotfun_f3pgercon + devppimtotfun_f3pgercon + tejgtotfun_f3protsoc + dfgpimpiatotfun_f3protsoc + devppimtotfun_f3protsoc + dfgdevpiagtotfun_f3salud + devppimtotfun_f3salud + piagtotfun_f3san + pimgtotfun_f3san + tejgtotfun_f3san + devppimtotfun_f3san + devppimtotfun_f3trab + devppimtotfun_f3trans + piagtotfun_f3viv + tejgtotfun_f3viv + devppimtotfun_f3viv + devppimtotfun_f4agro + piagtotfun_f4amb + pimgtotfun_f4amb + tdvgtotfun_f4amb + tejgtotfun_f4cydep + dfgdevpiagtotfun_f4cydep + devppimtotfun_f4cydep + tejgtotfun_f4energia + dfgpimpiatotfun_f4energia + dfgdevpiagtotfun_f4energia + devppimtotfun_f4energia + tejgtotfun_f4opseg + devppimtotfun_f4opseg + piagtotfun_f4pgercon + tejgtotfun_f4pgercon + devppimtotfun_f4pgercon + dfgpimpiatotfun_f4prevsoc + tejgtotfun_f4protsoc + dfgdevpiagtotfun_f4protsoc + devppimtotfun_f4protsoc + piagtotfun_f4salud + devppimtotfun_f4salud + devppimtotfun_f4san + tdvgtotfun_f4trab + devppimtotfun_f4trab + devppimtotfun_f4trans + dfgpimpiatotfun_f4turi + dfgdevpiagtotfun_f4turi + devppimtotfun_f4turi + devppimtotfun_f4viv + piagtotfun_f5agro + devppimtotfun_f5agro + tejgtotfun_f5amb + devppimtotfun_f5amb + pimgtotfun_f5come + tejgtotfun_f5come + pimgtotfun_f5comunica + piagtotfun_f5cydep + pimgtotfun_f5cydep + tejgtotfun_f5cydep + devppimtotfun_f5cydep + piagtotfun_f5dpub + pimgtotfun_f5dpub + tejgtotfun_f5dpub + devppimtotfun_f5dpub + tejgtotfun_f5edu + devppimtotfun_f5ind + piagtotfun_f5opseg + devppimtotfun_f5opseg + devppimtotfun_f5prevsoc + piagtotfun_f5protsoc + tejgtotfun_f5protsoc + tdvgtotfun_f5protsoc + devppimtotfun_f5protsoc + piagtotfun_f5r07agro + devppimtotfun_f5r07agro + pimgtotfun_f5r07amb + devppimtotfun_f5r07amb + piagtotfun_f5r07come + tejgtotfun_f5r07come + devppimtotfun_f5r07come + piagtotfun_f5r07comunica + tejgtotfun_f5r07comunica + devppimtotfun_f5r07comunica + devppimtotfun_f5r07cydep + piagtotfun_f5r07dpub + pimgtotfun_f5r07dpub + piagtotfun_f5r07edu + pimgtotfun_f5r07edu + pimgtotfun_f5r07energia + tejgtotfun_f5r07energia + piagtotfun_f5r07ind + piagtotfun_f5r07opseg + pimgtotfun_f5r07opseg + tejgtotfun_f5r07opseg + piagtotfun_f5r07pesca + tejgtotfun_f5r07pesca + pimgtotfun_f5r07pgercon + tejgtotfun_f5r07pgercon + tejgtotfun_f5r07prevsoc + devppimtotfun_f5r07prevsoc + piagtotfun_f5r07protsoc + tejgtotfun_f5r07protsoc + tdvgtotfun_f5r07protsoc + tejgtotfun_f5r07salud + devppimtotfun_f5r07salud + piagtotfun_f5r07san + pimgtotfun_f5r07san + tejgtotfun_f5r07san + devppimtotfun_f5r07san + devppimtotfun_f5r07trab + pimgtotfun_f5r07turi + pimgtotfun_f5r07viv + tejgtotfun_f5r07viv + devppimtotfun_f5r07viv + piagtotfun_f5r08agro + tejgtotfun_f5r08agro + tejgtotfun_f5r08amb + tejgtotfun_f5r08come + pimgtotfun_f5r08cydep + pimgtotfun_f5r08edu + devppimtotfun_f5r08edu + piagtotfun_f5r08energia + tejgtotfun_f5r08energia + devppimtotfun_f5r08energia + piagtotfun_f5r08ind + piagtotfun_f5r08opseg + tejgtotfun_f5r08opseg + pimgtotfun_f5r08pgercon + devppimtotfun_f5r08prevsoc + pimgtotfun_f5r08protsoc + tejgtotfun_f5r08protsoc + pimgtotfun_f5r08salud + devppimtotfun_f5r08salud + piagtotfun_f5r08san + tejgtotfun_f5r08trab + devppimtotfun_f5r08trab + devppimtotfun_f5r08turi + devppimtotfun_f5r08viv + pimgtotfun_f5r18agro + devppimtotfun_f5r18agro + devppimtotfun_f5r18amb + piagtotfun_f5r18come + devppimtotfun_f5r18come + piagtotfun_f5r18cydep + devppimtotfun_f5r18cydep + tejgtotfun_f5r18dpub + devppimtotfun_f5r18dpub + devppimtotfun_f5r18edu + piagtotfun_f5r18ind + piagtotfun_f5r18opseg + tejgtotfun_f5r18opseg + devppimtotfun_f5r18opseg + piagtotfun_f5r18pesca + devppimtotfun_f5r18pgercon + devppimtotfun_f5r18prevsoc + tdvgtotfun_f5r18protsoc + devppimtotfun_f5r18protsoc + devppimtotfun_f5r18salud + devppimtotfun_f5r18san + tejgtotfun_f5r18trab + devppimtotfun_f5r18trab + devppimtotfun_f5r18trans + tejgtotfun_f5r18turi + devppimtotfun_f5r18turi + piagtotfun_f5r18viv + pimgtotfun_f5r18viv + devppimtotfun_f5r18viv + piagtotfun_f5salud + tdvgtotfun_f5salud + devppimtotfun_f5salud + piagtotfun_f5trab + pimgtotfun_f5trab + devppimtotfun_f5trab + tejgtotfun_f5trans + tdvgtotfun_f5turi + devppimtotfun_f5turi + piagtotfun_f5viv + tejgtotfun_f5viv + tdvgtotfun_f5viv + devppimtotfun_f5viv"
y_basic_train, x_basic_train = patsy.dmatrices( formula_basic, data_train, return_type = 'dataframe' )
y_basic_test, x_basic_test = patsy.dmatrices( formula_basic, data_test, return_type = 'dataframe' )
p_basic = x_basic_train.shape[ 1 ]

Wall time: 2.03 s


#### 2.2. Generar variables dependientes

In [11]:
Y_train = data_train[ 'per_corrup2' ]
Y_test = data_test[ 'per_corrup2' ]

In [12]:
p_basic

826

## 3. OLS

In [13]:
x_basic_train.head()

,Intercept,tejgfun_ct05pgercon,devppimfun_ct05pgercon,devppimfun_ct05opseg,devppimfun_ct05trab,piagfun_ct05come,devppimfun_ct05turi,tejgfun_ct05agro,tdvgfun_ct05agro,devppimfun_ct05agro,...,piagtotfun_f5trab,pimgtotfun_f5trab,devppimtotfun_f5trab,tejgtotfun_f5trans,tdvgtotfun_f5turi,devppimtotfun_f5turi,piagtotfun_f5viv,tejgtotfun_f5viv,tdvgtotfun_f5viv,devppimtotfun_f5viv
366,1.0,14.607135,4.540876,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,6.746412,0.0,0.0,0.000000,10.221341,10.221341,4.137062
266,1.0,13.113903,4.297514,0.000000,0.0,0.000000,4.199780,0.0,0.00000,0.000000,...,0.0,0.0,0.0,10.449613,0.0,0.0,0.000000,11.192161,11.192161,4.248103
156,1.0,0.000000,4.544637,0.000000,0.0,0.000000,0.000000,0.0,8.03948,4.615121,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000
572,1.0,15.373755,4.488539,0.000000,0.0,0.000000,2.941129,0.0,0.00000,0.000000,...,0.0,0.0,0.0,14.663879,0.0,0.0,0.000000,11.606425,11.606425,4.615121
187,1.0,15.973295,4.525001,4.476505,0.0,13.335306,0.000000,0.0,0.00000,0.000000,...,0.0,0.0,0.0,16.525196,0.0,0.0,15.303035,15.029373,15.029313,4.574586


#### 3.1. Ajustar el modelo

In [14]:
fit_lm_basic = smf.ols( formula_basic, data = data_train ).fit()

#### 3.2. Calcular out-of-sample MSE y Standard Error

In [15]:
# Only MSE

yhat_lm_basic = fit_lm_basic.predict( data_test )
print("The mean squared error (MSE) using the basic model is equal to:", ( ( Y_test - yhat_lm_basic )**2 ).mean() )

The mean squared error (MSE) using the basic model is equal to: 95.49101759718765


In [16]:
# MSE and SE

resid_basic  = ( Y_test - yhat_lm_basic )**2

MSE_lm_basic = sm.OLS( resid_basic , np.ones( resid_basic.shape[ 0 ] ) ).fit().summary2().tables[ 1 ].iloc[ 0, 0:2 ]
MSE_lm_basic

Coef.       95.491018
Std.Err.    11.558104
Name: const, dtype: float64

#### 3.3. Calcular R Cuadrado out-of-sample

In [17]:
R2_lm_basic = 1 - ( MSE_lm_basic[ 0 ]/Y_test.var() )
print( f"The R^2 using the basic model is equal to: { R2_lm_basic }" )

The R^2 using the basic model is equal to: -5.765300686504828


## 4. Lasso, Ridge and Elastic Net

#### 4.1. Theoretical Lasso from hdm package

In [18]:
import hdmpy as hdm

In [19]:
fit_rlasso = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = False )
fit_rlasso_post = hdm.rlasso( x_basic_train.to_numpy() , Y_train.to_numpy().reshape( Y_train.size , 1 ) , post = True )

In [20]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape( x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    ( np.ones( ( x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T )

# Getting the significant variables
x1_est_rlasso = new_x1[ :, fit_rlasso.est[ 'index' ].iloc[ :, 0 ].to_list() ]

# Convert fit_rlasso.est[ 'beta' ] to dataframe
fit_rlasso_est = pd.DataFrame( fit_rlasso.est[ 'beta' ] )

# Getting the coef. from significant variables
beta_rlasso = fit_rlasso_est.loc[ fit_rlasso.est[ 'index' ].\
                                  loc[ :, 0 ].to_list(), ].to_numpy()

# yhat
yhat_rlasso = ( x1_est_rlasso @ beta_rlasso ) + np.mean( Y_test.to_numpy() )
residuals_rlasso = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso

In [21]:
# Getting mean of each variable
meanx = x_basic_test.mean( axis = 0 ).values.\
                        reshape(x_basic_test.shape[ 1 ] , 1 )

# Reducing the mean
new_x1 = x_basic_test.to_numpy() - \
                    (np.ones( (x_basic_test.shape[ 0 ] , 1 ) ) @ meanx.T)

# Getting the significant variables
x1_est_rlasso_post = new_x1[ :, fit_rlasso_post.est['index'].iloc[:, 0].to_list()]

# Convert fit_rlasso.est[ 'beta' ] to dataframe
fit_rlasso_post_est = pd.DataFrame( fit_rlasso_post.est[ 'beta' ] )

# Getting the coef. from significant variables
beta_rlasso_post = fit_rlasso_post_est.loc[ fit_rlasso_post.est['index'].\
                                            iloc[:, 0].to_list(), ].to_numpy()

# yhat
yhat_rlasso_post = (x1_est_rlasso_post @ beta_rlasso_post) + np.mean( Y_test.to_numpy() )
residuals_rlasso_post = Y_test.to_numpy().reshape( Y_test.to_numpy().size, 1)  - yhat_rlasso_post

In [22]:
MSE_lasso = sm.OLS( ( residuals_rlasso )**2 , np.ones( yhat_rlasso.size )  ).fit().summary2().tables[1].round(3)
MSE_lasso_post = sm.OLS( ( residuals_rlasso_post )**2  , np.ones( yhat_rlasso_post.size )  ).fit().summary2().tables[1].round(3)

R2_lasso = 1 - MSE_lasso.iloc[0, 0]/ np.var( Y_test )
R2_lasso_post = 1 - MSE_lasso_post.iloc[0, 0]/ np.var( Y_test )

print( f"The R^2 using the basic model is equal to {R2_lasso},for lasso and {R2_lasso_post} for post-lasso")

The R^2 using the basic model is equal to 0.007572473411392111,for lasso and -0.01124672404726268 for post-lasso


#### 4.2. Cross Validated Lasso, Ridge and Elastic Net

In [23]:
from sklearn.linear_model import LassoCV, RidgeCV, ElasticNetCV

In [24]:
fit_lasso_cv = LassoCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )
fit_ridge = RidgeCV( cv = 10, fit_intercept = True, normalize = False, scoring = None ).fit( x_basic_train, Y_train )
fit_elnet = ElasticNetCV( cv = 10, fit_intercept = True, normalize = False, random_state = 0 ).fit( x_basic_train, Y_train )

yhat_lasso_cv = fit_lasso_cv.predict( x_basic_test )
yhat_ridge = fit_ridge.predict( x_basic_test )
yhat_elnet = fit_elnet.predict( x_basic_test )

residual_lasso = ( yhat_lasso_cv - Y_test )**2
residual_ridge = ( yhat_ridge - Y_test )**2
residual_elnet = ( yhat_elnet - Y_test )**2

MSE_lasso_cv = sm.OLS( residual_lasso, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_ridge = sm.OLS( residual_ridge, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_elnet = sm.OLS( residual_elnet, np.ones( Y_test.size )).fit().summary2().tables[ 1 ].round( 3 )

R2_lasso_cv = 1 - MSE_lasso_cv.iloc[ 0, 0 ] / np.var( Y_test )
R2_ridge = 1 - MSE_ridge.iloc[ 0, 0 ]  / np.var( Y_test )
R2_elnet = 1 - MSE_elnet.iloc[ 0, 0 ]  / np.var( Y_test )

print( "R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: {:.5f}, {:.5f}, {:.5f}".format( R2_lasso_cv, R2_ridge, R2_elnet ) )

R^2 using cross-validation for lasso, ridge, and elastic net in the basic model: -0.01032, -4.15625, -0.00968


## 5. Non Linear Models

#### 5.1. Regression Trees

In [25]:
from sklearn.tree import DecisionTreeRegressor
from sklearn import tree

In [26]:
# Fit the model

fit_trees = DecisionTreeRegressor( random_state = 0, min_impurity_decrease = 0.001 )
fit_trees.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(min_impurity_decrease=0.001, random_state=0)

In [27]:
# Plot the tree

# from sklearn.tree import plot_tree

# plt.figure( figsize=( 30, 20 ) )
# plot_tree( fit_trees, filled = True )
# plt.show()

In [28]:
# Determine the optimar complexity of the regression tree

s = pd.DataFrame( fit_trees.cost_complexity_pruning_path( y_basic_train, x_basic_train ) )
s.head()

,ccp_alphas,impurities
0,0.000000,9.034151
1,0.016901,9.067952
2,0.017186,9.102325
3,0.017838,9.120162
4,0.017981,9.138143


In [29]:
### PREGUNTA: DE DÓNDE SALE ESE VALOR DE ALPHA

In [30]:
# Prune the tree

fit_prunnedtree = DecisionTreeRegressor( ccp_alpha = 0.00188444410871555 )
fit_prunnedtree.fit( x_basic_train, y_basic_train )

DecisionTreeRegressor(ccp_alpha=0.00188444410871555)

In [31]:
# Plot the prunned tree

# plot_tree(fit_prunnedtree, filled=True)
# plt.show()

In [32]:
# Calculate MSE and R2 for prunned tree

y_hat_pt = fit_prunnedtree.predict( x_basic_test )
residual_pt = ( y_hat_pt - Y_test )**2
MSE_pt = sm.OLS( residual_pt, np.ones( y_hat_pt.size )).fit().summary2().tables[ 1 ].round( 3 )
R2_pt = 1 - MSE_pt.iloc[ 0, 0 ]/np.var( Y_test )
print( f"R^2 of the pruned tree: { R2_pt }" )

R^2 of the pruned tree: -1.2118972649416686


#### 5.2. Random Forest and Booested Trees

In [33]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor 

In [34]:
# random forest
fit_rf = RandomForestRegressor( n_estimators = 2000, min_samples_leaf = 5).fit( x_basic_train, Y_train )

# boosting
fit_boost = GradientBoostingRegressor( loss = 'ls', learning_rate = 0.01, n_estimators = 1000, max_depth = 2, subsample = 0.5).fit( x_basic_train, Y_train )

# Evaluating the methods
yhat_rf = fit_rf.predict( x_basic_test )
residual_rf = ( yhat_rf - Y_test )**2
yhat_boost = fit_boost.predict( x_basic_test )
residual_bst = ( yhat_boost - Y_test )**2

# Calculate MSE
MSE_rf = sm.OLS( residual_pt, np.ones( yhat_rf.size )).fit().summary2().tables[ 1 ].round( 3 )
MSE_bst = sm.OLS( residual_bst, np.ones( yhat_boost.size )).fit().summary2().tables[ 1 ].round( 3 )

# Calculate R2
R2_rf = 1 - MSE_rf.iloc[ 0, 0 ] / Y_test.var()
R2_boost = 1 - MSE_bst.iloc[ 0, 0 ] / Y_test.var()

In [35]:
print("R^2 of the random forest and boosted trees:{:.5f}, {:.5f}".format( R2_rf, R2_boost ))

R^2 of the random forest and boosted trees:-1.19833, -0.22184


## 6. Resultados

In [36]:
table = pd.DataFrame(columns=["MSE", "S.E for MSE", "R-squared"]) 
table.loc[0]  = [MSE_lm_basic[0], MSE_lm_basic[1], R2_lm_basic]
table.loc[1]  = [MSE_lasso.iloc[0, 0], MSE_lasso.iloc[0, 1], R2_lasso]
table.loc[2]  = [MSE_lasso.iloc[0, 0], MSE_lasso_post.iloc[0, 1], R2_lasso_post]
table.loc[3]  = [MSE_lasso_cv.iloc[0, 0], MSE_lasso_cv.iloc[0, 1], R2_lasso_cv]
table.loc[4]  = [MSE_ridge.iloc[0, 0], MSE_ridge.iloc[0, 1], R2_ridge]
table.loc[5]  = [MSE_elnet.iloc[0, 0], MSE_elnet.iloc[0, 1], R2_elnet]
table.loc[6] = [MSE_rf.iloc[0, 0], MSE_rf.iloc[0, 1], R2_rf]
table.loc[7] = [MSE_bst.iloc[0, 0], MSE_bst.iloc[0, 1], R2_boost]
table.loc[8] = [MSE_pt.iloc[0, 0], MSE_pt.iloc[0, 1], R2_pt]
models_row = [ "Least Squares (basic)", "Lasso", "Post_Lasso", "Cross-Validated lasso", 
               "Cross-Validated ridge", "Cross-Validated elnet", "Random Forest", 
               "Boosted Trees", "Pruned Tree" ]
table.insert( 0, "Models", models_row )
table

,Models,MSE,S.E for MSE,R-squared
0,Least Squares (basic),95.491018,11.558104,-5.765301
1,Lasso,13.922000,3.346000,0.007572
2,Post_Lasso,13.922000,3.401000,-0.011247
3,Cross-Validated lasso,14.173000,3.321000,-0.010320
4,Cross-Validated ridge,72.333000,8.783000,-4.156246
5,Cross-Validated elnet,14.164000,3.314000,-0.009678
6,Random Forest,31.029000,5.144000,-1.198327
7,Boosted Trees,17.246000,3.362000,-0.221836
8,Pruned Tree,31.029000,5.144000,-1.211897


Apuntes:

* Para definir los modelos patsy, estos borran por default la data missing. Para evitar este comportamiento se adiciona el parámetro: NA_action=patsy.NAAction(NA_types=[])
https://stackoverflow.com/questions/51640071/stop-patsy-dmatrix-from-dropping-nan-rows


* Para correr con missing values los modelos OLS, se adiciona el parámetro: missing='drop'

* Si ponemos los dos ajustes, el modelo OLS si corre. 

* Este error "zero-size array to reduction operation maximum which has no identity" ocurre porque cuando tenemos demasiados missings, la dataframe se queda sin columnas, pues todas tienen al menos un missing. https://stackoverflow.com/questions/45545774/multiple-ols-regression-with-statsmodel-valueerror-zero-size-array-to-reduction